# Data Pre-Processing - Gradient Boosting Classifier Model

In [7]:

import pandas as pd

df = pd.read_csv("/workspaces/CS_Group07_StudentDepressionDetection/notebooks/menura-gradient_boosting_classifier/cleaned_student_depression(GBC-model).csv")

categorical_cols = df.select_dtypes(include='object').columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

df.head()

,Gender,Age,Academic_Pressure,CGPA,Study_Satisfaction,Sleep_Duration,Dietary_Habits,Degree,Suicidal Thoughts,Work/Study_Hours,Financial_Stress,Mental Illness History,Depression
0,Male,33.0,5.0,8.97,2.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,Female,24.0,2.0,5.90,5.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,Male,31.0,3.0,7.03,5.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,Female,28.0,3.0,5.59,2.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,Female,25.0,4.0,8.13,3.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


## One-Hot Encoding

In [8]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_encoded.head()


,Age,Academic_Pressure,CGPA,Study_Satisfaction,Work/Study_Hours,Financial_Stress,Depression,Gender_Male,Sleep_Duration_7-8 hours,Sleep_Duration_Less than 5 hours,...,Degree_MBBS,Degree_MCA,Degree_MD,Degree_ME,Degree_MHM,Degree_MSc,Degree_Others,Degree_PhD,Suicidal Thoughts_Yes,Mental Illness History_Yes
0,33.0,5.0,8.97,2.0,3.0,1.0,1,True,False,False,...,False,False,False,False,False,False,False,False,True,False
1,24.0,2.0,5.90,5.0,3.0,2.0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,31.0,3.0,7.03,5.0,9.0,1.0,0,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,28.0,3.0,5.59,2.0,4.0,5.0,1,False,True,False,...,False,False,False,False,False,False,False,False,True,True
4,25.0,4.0,8.13,3.0,1.0,1.0,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False


## Scale Numerical Features

In [9]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])
df_encoded.head()


,Age,Academic_Pressure,CGPA,Study_Satisfaction,Work/Study_Hours,Financial_Stress,Depression,Gender_Male,Sleep_Duration_7-8 hours,Sleep_Duration_Less than 5 hours,...,Degree_MBBS,Degree_MCA,Degree_MD,Degree_ME,Degree_MHM,Degree_MSc,Degree_Others,Degree_PhD,Suicidal Thoughts_Yes,Mental Illness History_Yes
0,1.463175,1.345717,0.893329,-0.693304,-1.121515,-1.488607,0.841961,True,False,False,...,False,False,False,False,False,False,False,False,True,False
1,-0.371146,-0.825658,-1.193957,1.510701,-1.121515,-0.792840,-1.187704,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1.055548,-0.101866,-0.425673,1.510701,0.497014,-1.488607,-1.187704,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,0.444108,-0.101866,-1.404725,-0.693304,-0.851760,1.294461,0.841961,False,True,False,...,False,False,False,False,False,False,False,False,True,True
4,-0.167332,0.621925,0.322215,0.041364,-1.661025,-1.488607,-1.187704,False,False,False,...,False,False,False,False,False,False,False,False,True,False


## Split Features (X) and Target (y)

In [11]:

X = df_encoded.drop("Depression", axis=1) 
y = df_encoded["Depression"]


## Split - Training & Testing sets (80-20 split)

In [12]:

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ModuleNotFoundError: No module named 'imblearn'

### SMOTE to training set - Balance the data set

In [ ]:
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

## Save Pre-processed Data Set

In [ ]:
df.to_csv("cleaned_student_depression(GBC-model).csv", index=False)